[View in Colaboratory](https://colab.research.google.com/github/vivek6449/Building_Damage_Classification/blob/master/Models.ipynb)

In [54]:
import tensorflow as tf 
tf.test.gpu_device_name() 

'/device:GPU:0'

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 

gpg: keybox '/tmp/tmpd8k9yb9d/pubring.gpg' created
gpg: /tmp/tmpd8k9yb9d/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
from google.colab import auth 
auth.authenticate_user() 

In [0]:
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 

In [0]:
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive 

In [7]:
print ('Files in Drive:')  
!ls drive/ 

Files in Drive:
Colab Notebooks      LSTM			Study
Docs		     Presentation.pdf		Untitled presentation.pdf
Job Description.odt  Resumes
Job Tracker.ods      Rotor Crack Detection.odt


In [8]:
!pip install lightgbm

In [0]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMClassifier

import gc

In [0]:
train = pd.read_csv('drive/Colab Notebooks/Building/train_full.csv')
test = pd.read_csv('drive/Colab Notebooks/Building/test_full.csv')

In [0]:
for_ids= pd.read_csv('drive/Colab Notebooks/Building/combined_train.csv')
building_id = for_ids['building_id']

In [31]:
train = pd.concat([building_id, train], axis=1)

,building_id,damage_grade,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,...,Damaged-Repaired and used,Damaged-Rubble Clear-New building built,Damaged-Rubble clear,Damaged-Rubble unclear,Damaged-Used in risk,Not damaged,Institutional,Other.2,Private,Public
0,24385bfd2a2,Grade 4,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,405d1bbebbf,Grade 2,0.0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,351d9bc71f6,Grade 1,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2be3a971166,Grade 5,0.0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,34c7d073ea6,Grade 3,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [0]:
y = train[['damage_grade']]
del train['damage_grade']

In [36]:
y['damage_grade'] = y['damage_grade'].str.replace('Grade ','')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
pd.to_numeric(y['damage_grade']);

In [0]:
from sklearn.ensemble import RandomForestClassifier

rnd_forest = RandomForestClassifier(n_estimators=100, random_state=42, verbose=1, n_jobs=-1)

In [0]:
train.set_index('building_id', inplace=True)

In [57]:
test.head()

,building_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,...,Damaged-Repaired and used,Damaged-Rubble Clear-New building built,Damaged-Rubble clear,Damaged-Rubble unclear,Damaged-Used in risk,Not damaged,Institutional,Other.2,Private,Public
0,a3380c4f75,0.0,0,0,0,0,0,0,0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,a338a4e653,0.0,0,0,0,0,0,0,0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,a338a4e6b7,0.0,0,0,0,0,0,0,0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,a33a6eaa3a,0.0,0,0,0,0,0,0,0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,a33b073ff6,0.0,0,0,0,0,0,0,0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [0]:
test.set_index('building_id', inplace=True)

In [49]:
rnd_forest.fit(train, y)
y_pred = rnd_forest.predict(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.6min finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    5.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   11.7s finished


In [0]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(max_depth=3, n_estimator=300, learning_rate=0.03)

In [59]:
xgb_clf.fit(train,y)
y_pred_xgb = xgb_clf.predict(test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
test.reset_index(inplace=True)

In [61]:
submit = test[['building_id']]
submit['damage_grade'] = y_pred_xgb

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [62]:
submit['damage_grade'] = "Grade " + submit['damage_grade'].astype(str)
submit.to_csv("drive/Colab Notebooks/Building/xgb_baseline.csv", index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
